In [3]:
import os
import json
import pandas as pd
import traceback

In [18]:
from dotenv import load_dotenv
load_dotenv()

True

In [7]:
from langchain_openai import ChatOpenAI

In [20]:
KEY = os.getenv("OPENAI_API_KEY")

In [21]:
llm = ChatOpenAI(openai_api_key=KEY,model_name="gpt-3.5-turbo",temperature=0.5)

In [22]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [24]:
RESPONSE_JSON = {
    "1": {
        "question":"multiple choice question",
        "options":{
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here"   
        },
        "correct":"correct_answer"
    },
    "2": {
        "question":"multiple choice question",
        "options":{
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here"   
        },
        "correct":"correct_answer"
    },
    "3": {
        "question":"multiple choice question",
        "options":{
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here"   
        },
        "correct":"correct_answer"
    },
}

In [25]:
TEMPLATE = """
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to create a quiz of {number} multiple choice questions for {subject} subject with {tone} tone.
Make sure that the questions should not be repeated and check all the questions conforming the text as well.
Make sure you format your response like RESPONSE_JSON below and use it as a guide.
Ensure to make {number} MCQs.
### RESPONSE_JSON
{response_json}
"""

In [26]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text","subject","number","tone","response_json"],
    template=TEMPLATE
)

In [27]:
quiz_chain = LLMChain(llm=llm,prompt=quiz_generation_prompt,output_key="quiz",verbose=True)

In [29]:
EVALUATION_TEMPLATE = """
You are an expert grammarian and writer. Given a multiple choice quiz for {subject} subject.
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for the analysis. 
If the quiz is not as per the cognitive and thinkig ability of the student then update the quiz questions that need to be changed and change the tone such that it perfectly fits the student abilities.
Quiz_MCQs:
{quiz}
"""

In [30]:
quiz_evaluation_prompt = PromptTemplate(input_variables=["subject","quiz"],template=EVALUATION_TEMPLATE)

In [32]:
review_chain = LLMChain(llm=llm,prompt=quiz_evaluation_prompt,output_key="review",verbose=True)

In [34]:
generate_quiz_chain = SequentialChain(chains=[quiz_chain,review_chain],input_variables=["text","subject","number","tone","response_json"],output_variables=["quiz","review"],verbose=True)